## Re-run the simulations

The basis parameters settings are in 3 files: "LBM_binary.H", "LBM_d3q19.H" and "main_run_job.cpp"; There are mainly $3$ cases: 

I. Mixture system;

II. Flat interface;

III. 3D Droplet;

In file "LBM_binary.H", on the top there are $3$ macros which define the above 3 cases. Comment out the other two macros and keep only one case, i.e.,   

                                #define SYS_DROPLET     # this is for Case III;
                                //#define SYS_MIXTURE
                                //#define SYS_FLATE_INTERFACE

### Case I. Mixture system

                    //#define SYS_DROPLET
                    #define SYS_MIXTURE
                    //#define SYS_FLATE_INTERFACE

By default, box size is $32\times32\times32$ and equilibrium state density: $\rho_A=\rho_B=1.0$ which is controlled by

                    const Real C1 = 0.5; // In function "LBM_init_mixture(*)", "LBM_binary.H"
                    const Real C2 = 0.5; 

##### Case A: Non-fluctuating case:
In "main_run_job.cpp" file,

                    int nx = 32
                    nsteps = 500 // for alpha0=0 (ideal gas)
                    step_continue = 0
                    plot_int = 10 
                    t_window = 10*plot_int; // last 400~500 steps (interval = plot_int, total 10+1 frames) are used 
In "LBM_binary.H" file,

                    alpha0 = 0. // ideal gas, change if necessary;
                    alpha1 = 0.
In "LBM_d3q19.H" file,

                    AMREX_GPU_MANAGED Real kBT = 0.;    // no thermal fluctuations;

##### Case B: Fluctuating case:
In "main_run_job.cpp" file,

                    nsteps = 600000 // for alpha0=0 (ideal gas)
                    step_continue = 500
                    plot_int = 2000 // output configurations every 2000 steps;
In "LBM_binary.H" file,

                    alpha0 = 0. // ideal gas, change if necessary;
                    alpha1 = 0.
In "LBM_d3q19.H" file,

                    AMREX_GPU_MANAGED Real kBT = 1e-5;    // no thermal fluctuations;
                    plot_SF_window = 200000     // time window for calculating the structure factor;
                    out_SF_step = 100   //select every 100 steps to calcualte Structure Factor;
Here it means that we choose last 200000 steps to calculate the Structure Factor using FHDeX.


### Case II. Flat interface

box size (8, 256, 64), initial density profile: rho_lo = 0.1, rho_hi = 3.   kappa = 0.1

##### Case A: Non-fluctuating case:
                alpha0 = 1.5
                alpha1 = 0.
                nsteps = 3000
                step_continue = 0
                plot_int = 10 
                t_window = 50*plot_int;

##### Case B: Fluctuating case:
                nsteps = 800000
                step_continue = 3000
                plot_int = 1000 // output configurations every 1000 steps;
                plot_SF_window = 0  // Do not use FHDeX lib to calculate the StructFactor, so set it to be 0;

### Case III. 3D Droplet

                                #define SYS_DROPLET
                                //#define SYS_MIXTURE
                                //#define SYS_FLATE_INTERFACE
##### Case A: Non-fluctuating case:
In "main_run_job.cpp" file,

                int nx = 64;
                int max_grid_size = nx/2; 
                int step_continue = 0;
                bool continueFromNonFluct = true;

                int nsteps = 30000; // Total number of steps to run; MUST be integer multiples of [plot_int];
                int plot_int = 50;  // output configurations to disk every [plot_int] steps;
                int print_int = 20; // print out info every [print_int] steps; have no effects on system;
                const int t_window = 5*plot_int;  //set as multiples of [plot_int], 
                            from step [last_step_index-t_window] to [last_step_index];
                const Real radius = 0.25;       // init droplet fractional radius magnitude;
                bool if_print_radius = false;   //print fitted droplet radius for each output frame by AMReX;
In "LBM_d3q19.H" file,

                AMREX_GPU_MANAGED Real kBT = 0.;    // no thermal fluctuations;
In "LBM_binary.H" file, we adjust the Shan-Chen model parameters:

                AMREX_GPU_MANAGED Real tau_f = 1./2.; // tau=0.5*(1.+1./sqrt(3.)) minimizes spurious currents according to Swift et al.
                AMREX_GPU_MANAGED Real tau_g = 1./2.;
                AMREX_GPU_MANAGED Real alpha0 = 4.; // alpha0 in Belardinelli's paper;
                AMREX_GPU_MANAGED Real alpha1 = 0.; //alpha1 in Belardinelli's paper;

                const bool use_SC_pseudo = false; // false: use density field directly; true: use SC pseudopotentials;
                const Real SC_ref_density = 1.0; // reference density for SC pseudopotential function (in LB3D =1)
                const Real rho_lo = 0.;     // control the init droplet profile, see function [LBM_init_droplet];
                const Real rho_hi = 1.0;    
                AMREX_GPU_MANAGED Real kappa = 4; // control the init interface thickness;

##### Case B: Fluctuating case:
In "main_run_job.cpp" file,

                int nx = 64;
                int max_grid_size = nx/2; 
                int step_continue = 30000; // by default keep it to be same with the [nsteps] in above non-fluctuating case;
                bool continueFromNonFluct = true;
Here have 2 choices: (1): true: the checkpoint file is the last frame output by Case A; switch on the noise term given the initial configuration specified by file "f_checkpoint[step]\_alpha0\_\*\_xi\_\*\_size\*-\*-\*"; (2) false: the checkpoint file is the last frame output by Case B, i.e., re-starting from fluctuating droplet trajectory.

                int nsteps = 200000;    // change it if needed; 
                int plot_int = 50;  // output configurations to disk every [plot_int] steps;
                int print_int = 20; // print out info every [print_int] steps; have no effects on system;
                bool if_print_radius = false;   //print fitted droplet radius for each output frame by AMReX;
In "LBM_d3q19.H" file,

                AMREX_GPU_MANAGED Real kBT = 5*1e-5;    // switch on thermal fluctuations;




illustration for the analysis code files;
specify which file is the analysis file;

email + git

usage about yt, how to use dictionary to read the data

### All users can access the data 

        chmod -R a+rwX /scratch/Binary-Fluctuating-Lattice-Boltzmann

        xdengae@oberon:~$ ls -ld /scratch/Binary-Fluctuating-Lattice-Boltzmann